# Generate Activity Type Icons

This notebook is used to generate the icons and definitions for the `#/$defs$defs/activityType` schema of the Semantic Location Data schema.

Icons source: [Material Icons: Google Fonts](https://fonts.google.com/icons).

In [ ]:
import json
from pathlib import Path

import pandas as pd
import xml.etree.ElementTree as ET

sys.path.append("../")
from common import download_text, save_text

In [ ]:
OUTPUT_PATH = "../../docs/static/icons/activity_types/"


# This data has been obtained from reverse engineering the Javascript in the Timeline website
# and generating test Google Takeout extractions that exhaustively covered all of the activities.
ICONS = [
    [0, "#03a9f4", "UNKNOWN_ACTIVITY_TYPE", "Moving", "moving"],
    [1, "#01579b", "STILL", "Still", "man"],  # Assigning 'STILL' to 1 is just a guess, and 1 is not taken otherwise.
    [2, "#03a9f4", "WALKING", "Walking", "directions_walk"],
    [3, "#4db6ac", "CYCLING", "Cycling", "directions_bike"],
    # [4 , "", "IN_VEHICLE", "", ""],  # Seems it is deprecated.
    [5, "#3f51b5", "FLYING", "Flying", "local_airport"],
    [6, "#c2185b", "RUNNING", "Running", "directions_run"],
    [7, "#01579b", "IN_BUS", "On a bus", "directions_bus"],
    [8, "#01579b", "IN_TRAIN", "On a train", "directions_railway"],
    [9, "#01579b", "IN_SUBWAY", "On the subway", "subway"],
    [10, "#01579b", "IN_TRAM", "On a tram", "tram"],
    [11, "#01579b", "IN_FERRY", "On a ferry", "directions_boat"],
    [12, "#01579b", "IN_CABLECAR", "In a cable car", "moving"],
    [13, "#01579b", "IN_FUNICULAR", "On a funicular", "moving"],
    [14, "#c2185b", "HIKING", "Hiking", "hiking"],
    [15, "#4db6ac", "KAYAKING", "Kayaking", "kayaking"],
    [16, "#4db6ac", "KITESURFING", "Kitesurfing", "kitesurfing"],
    [17, "#c2185b", "ROWING", "Rowing", "rowing"],
    [18, "#4db6ac", "SAILING", "Sailing", "sailing"],
    [19, "#4db6ac", "SKATING", "Skating", "ice_skating"],
    [20, "#4db6ac", "SKIING", "Skiing", "downhill_skiing"],
    [21, "#4db6ac", "SKATEBOARDING", "Skateboarding", "skateboarding"],
    [22, "#4db6ac", "SLEDDING", "Sledding", "sledding"],
    [23, "#4db6ac", "SNOWBOARDING", "Snowboarding", "snowboarding"],
    [24, "#01579b", "SNOWMOBILE", "Snowmobiling", "snowmobile"],
    [25, "#c2185b", "SNOWSHOEING", "Snowshoeing", "snowshoeing"],
    [26, "#4db6ac", "SURFING", "Surfing", "surfing"],
    [27, "#c2185b", "SWIMMING", "Swimming", "pool"],
    [28, "#c2185b", "WALKING_NORDIC", "Nordic walking", "nordic_walking"],
    [29, "#01579b", "IN_PASSENGER_VEHICLE", "Driving", "directions_car"],
    [30, "#01579b", "MOTORCYCLING", "Motorcycling", "motorcycle"],
    [31, "#01579b", "BOATING", "Boating", "directions_boat"],
    [32, "#03a9f4", "IN_WHEELCHAIR", "By wheelchair", "accessible_forward"],
    [33, "#4db6ac", "HORSEBACK_RIDING", "Horseback riding", "moving"],
    [34, "#01579b", "IN_GONDOLA_LIFT", "In a gondola lift", "moving"],
    [35, "#db4437", "CATCHING_POKEMON", "Catching Pokémon", "catching_pokemon"],
    [36, "#01579b", "IN_TAXI", "In a taxi", "local_taxi"],
    [37, "#4db6ac", "PARAGLIDING", "Paragliding", "paragliding"],
]

columns = ["activity_id", "activity_color", "activity_key", "activity_name", "material_icon"]

In [ ]:
df_activities = pd.DataFrame.from_records(ICONS, columns=columns)
df_activities

In [ ]:
def process_svg(svg_content):
    ns = {"svg": "http://www.w3.org/2000/svg"}
    ET.register_namespace("", ns["svg"])

    svg = ET.fromstring(svg_content)

    # Set the main color.
    svg.attrib["fill"] = "#666"

    # Return the SVG as a string.
    return ET.tostring(svg, method="xml", encoding="unicode")

## Generate Icons

In [ ]:
for idx, row in df_activities.iterrows():
    icon_name = row['material_icon']

    icon_urls = [
        f"https://fonts.gstatic.com/s/i/materialiconsround/{icon_name}/v4/24px.svg",
        f"https://fonts.gstatic.com/s/i/materialiconsround/{icon_name}/v12/24px.svg",
        f"https://fonts.gstatic.com/s/i/materialiconsround/{icon_name}/v14/24px.svg",
        f"https://fonts.gstatic.com/s/i/materialiconsround/{icon_name}/v17/24px.svg",
        f"https://fonts.gstatic.com/s/i/short-term/release/materialsymbolsrounded/{icon_name}/fill1/24px.svg",
    ]

    file_path = Path(OUTPUT_PATH) / f"{icon_name}_24dp.svg"

    if file_path.is_file():
        print(f"Skipping '{file_path}' (already exists)")
        continue

    for icon_url in icon_urls:
        content = download_text(icon_url)

        if content:
            break

    output_svg = process_svg(content)
    save_text(output_svg, file_path)
    print(f"Saved '{file_path}'")

## Generate Schema Text

In [ ]:
json_objs = []

for idx, row in df_activities.iterrows():
    json_objs.append(
        {
            "type": "string",
            "const": row['activity_key'],
            "title": row['activity_name'],
            "description": "",
            "extra_color": row['activity_color'],
            "extra_icon": f"activity_types/{row['material_icon']}_24dp.svg"
        }
    )

print(json.dumps(json_objs, indent=4))